# Deep 2D-CNN

# Preliminaries

Check for GPU

In [1]:
import tensorflow as tf

In [2]:
if tf.test.gpu_device_name() != '/device:GPU:0':
  print("No GPU found")
else:
  print("GPU ready: {}".format(tf.test.gpu_device_name()))

No GPU found


Import preprocessing helper functions

In [3]:
import sys
sys.path.append("../../helper-modules")
from preprocessing_utils import read_in_data, preprocess

Read in the data to df_train, df_val and df_test

In [4]:
df_train, df_val, df_test = read_in_data()

Create X_train, y_train, X_val, y_val, X_test, y_test

In [5]:
(X_train, y_train), (X_val, y_val), (X_test, y_test) = preprocess("CNN", df_train, df_test, df_val)

# Model Building

Imports

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, AvgPool2D, Flatten, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from itertools import product
import time
import matplotlib.pyplot as plt

Enable seed setting for improved reproducibility

In [7]:
import os
import random
import numpy as np

In [8]:
def set_seed(seed=42):
  SEED=seed

  # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
  os.environ['PYTHONHASHSEED']=str(SEED)

  # 2. Set `python` built-in pseudo-random generator at a fixed value
  random.seed(SEED)

  # 3. Set `numpy` pseudo-random generator at a fixed value
  np.random.seed(SEED)

  # 4. Set the `tensorflow` pseudo-random generator at a fixed value
  tf.random.set_seed(SEED)

Extract number of classes (10 classes)

In [9]:
NUM_CLASSES = df_test["label"].nunique() 

Model design

In [10]:
def compile_model(num_filters_tuple, filter_size = 3, padding_type="same", act_function="relu", pooling_type="max", dense_layer_size=16, dropout_rate=0.1, l_rate=0.001, reg='l2', opt_algo='adam'):
    """
    Compiles a deep 2D-CNN model with the given hyperparameters.
    
    Architecture:
    
    Conv layer -> Conv layer -> Pooling layer -> Conv layer -> Conv layer -> Pooling layer
    ->  Fully connected layer -> Softmax output layer
    
    Hyperparameter options:
    
    num_filters_tuple : A 4-tuple with the number of filters for each conv layer
    filter_size : an int with the size of the filters (3 -> 3 x 3 filter)
    padding_type : either "same" or "valid"
    act_function : options include "relu", "sigmoid" and "tanh"
    pooling_type : either "max" or "avg"
    dense_layer_size : an int with the number of neurons in the fully connected layer
    dropout_rate : a float with the rate of dropout to be applied to the fully connected layer
    l_rate : a float with the learning rate
    reg : string with the regularization type e.g. "l2"
    opt_algo : the optimization algorithm, "adam" or "sgd"
    """  
    model = Sequential()

    # CONVOLUTIONAL LAYER 1
    model.add(Conv2D(filters=num_filters_tuple[0], kernel_size=(filter_size,filter_size), input_shape=(40, 37, 1), padding=padding_type))
    model.add(Activation(act_function))

    # CONVOLUTIONAL LAYER 2
    model.add(Conv2D(filters=num_filters_tuple[1], kernel_size=(filter_size,filter_size), padding=padding_type))
    model.add(Activation(act_function))

    # POOLING LAYER 1
    if pooling_type == "avg":  # strides defaults to pool_size
      model.add(AvgPool2D(pool_size=(2, 2)))
    elif pooling_type == "max":
      model.add(MaxPool2D(pool_size=(2, 2)))
    else:
      raise Exception("Invalid pooling option entered")
      
    # CONVOLUTIONAL LAYER 3
    model.add(Conv2D(filters=num_filters_tuple[2], kernel_size=(filter_size,filter_size), padding=padding_type))
    model.add(Activation(act_function))

     # CONVOLUTIONAL LAYER 4
    model.add(Conv2D(filters=num_filters_tuple[3], kernel_size=(filter_size,filter_size), padding=padding_type))
    model.add(Activation(act_function))

    # POOLING LAYER 2
    if pooling_type == "avg":  # strides defaults to pool_size
      model.add(AvgPool2D(pool_size=(2, 2)))
    elif pooling_type == "max":
      model.add(MaxPool2D(pool_size=(2, 2)))
    else:
      raise Exception("Invalid pooling option entered")

    # FLATTEN OUTPUT
    model.add(Flatten())

    # # FULLY CONNECTED LAYER
    model.add(Dense(dense_layer_size, activation=act_function))

    # # DROPOUT to prevent overfitting
    model.add(Dropout(dropout_rate))
    
    # LAST LAYER IS THE CLASSIFIER, THUS 12 POSSIBLE CLASSES
    model.add(Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=reg))

    # Optimization algorithm
    if opt_algo == 'adam':
      opt = tf.keras.optimizers.Adam(learning_rate=l_rate)
    elif opt_algo == 'sgd':
      opt = tf.keras.optimizers.SGD(learning_rate=l_rate)

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

# Experiment: Grid Search

Defining grid search hyperparameter options

In [11]:
num_filters_tuples = [(4, 4, 4, 4), (8, 8, 8, 8), (16, 16, 16, 16), (32, 32, 32, 32), (64, 64, 64, 64), (128, 128, 128, 128)]
l_rates = [0.01, 0.005, 0.001, 0.0005, 0.0001]
dropout_rates = [0.05, 0.1, 0.2, 0.5]
pooling_types = ["max"]
filter_sizes = [3]
padding_types = ["same"]
act_functions=["relu"]
regs = ['l2']
opt_algos = ['adam']

all_hparams = [num_filters_tuples, l_rates, dropout_rates, pooling_types, filter_sizes, padding_types, act_functions, regs, opt_algos]

Conduct the grid search

In [12]:
def grid_search():
  """
  For each `num_filters_tuples` option (corresponding to a given number of parameters), conduct a grid search
  over the hyperparameter options, writing the validation results and model details for each model to a file.
  
  All trained models are saved as h5 files. 
  """
  best_model = None
  best_accuracy = 0
  best_history = None

  EPOCHS = 50
  BATCH_SIZE = 32
  STOPPING_PATIENCE = 5

  STARTING_POINT = 1

  with open('../training-results/ExperimentLogs_Deep-CNN.csv', 'w') as log_file:
      log_file.write("ModelNumber;Timestamp;NumEpochs;ValAccuracy;NumParams;NumFiltersPerLayer;DenseLayerSize;LearningRate;DropoutRate;PoolingType;FilterSize;PaddingType;ActivationFunction;Regularization;OptimizationAlgorithm\n")

  for model_num, hparam_set in enumerate(list(product(*all_hparams))[(STARTING_POINT-1):], start=STARTING_POINT):
    # Extract hyperparams for current model from grid search grid
    num_filters_tuple, l_rate, dropout_rate, pooling_type, filter_size, padding_type, act_function, reg, opt_algo = hparam_set
    dense_layer_size = num_filters_tuple[0]  # set number of dense layer neurons to number of filters of first two conv layers

    # Define hyperparameters for the current model 
    hparams = {
        "num_filters_tuple" : num_filters_tuple,
        "filter_size" : filter_size, 
        "padding_type" : padding_type, 
        "act_function" : act_function, 
        "pooling_type" : pooling_type, 
        "dense_layer_size" : dense_layer_size, 
        "dropout_rate" : dropout_rate, 
        "l_rate": l_rate,
        "reg" : reg,
        "opt_algo" : opt_algo
    }

    # Compile model and count number of parameters
    model = compile_model(**hparams)
    num_params = model.count_params()

    model_str = f"{num_filters_tuple}; {dense_layer_size}; {l_rate}; {dropout_rate}; {pooling_type}; {filter_size}; {padding_type}; {act_function}; {reg}; {opt_algo}"
    print(f"Training model {model_num} with {num_params} params - {model_str}")

    # Prevent overfitting with early stopping
    early_stop = EarlyStopping(monitor='val_accuracy', patience=STOPPING_PATIENCE)
    # For saving best val accuracy model as an h5 (for test predictions later)
    model_check = ModelCheckpoint(f"../trained-models/Deep-CNN_{model_num}.h5", monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)

    # Set seed for reproducibility
    set_seed(42)

    # Train model
    # Note: one reason val accuracy might be higher than train accuracy during training is because dropout affects training but not validation
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), callbacks=[early_stop, model_check])
    n_epochs = len(history.history['loss'])
                   
    # Time stamp when model finished training
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    # Extract best validation accuracy at inex 1 (index 0 has the loss)
    val_accuracy = max(history.history['val_accuracy'])
    print(f"Best val accuracy: {val_accuracy}")

    # Store results in log file
    with open('../training-results/ExperimentLogs_Deep-CNN.csv', 'a') as log_file:
      log_file.write(f"{model_num};{timestamp};{n_epochs};{val_accuracy};{num_params};{model_str}\n")

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      best_model = model
      best_history = history

  # Return the model with the highest validation accuracy
  return best_model, best_history


In [13]:
best_model, best_history = grid_search()

Training model 1 with 1993 params - (4, 4, 4, 4); 4; 0.01; 0.05; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 11s 4ms/step - loss: 2.6225 - accuracy: 0.0757 - val_loss: 2.6237 - val_accuracy: 0.0769
Epoch 2/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6229 - accuracy: 0.0758 - val_loss: 2.6235 - val_accuracy: 0.0769
Epoch 3/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6230 - accuracy: 0.0763 - val_loss: 2.6227 - val_accuracy: 0.0769
Epoch 4/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6230 - accuracy: 0.0761 - val_loss: 2.6225 - val_accuracy: 0.0769
Epoch 5/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6229 - accuracy: 0.0766 - val_loss: 2.6229 - val_accuracy: 0.0769
Epoch 6/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6231 - accuracy: 0.0769 - val_loss: 2.6223 - val_accuracy: 0.0769
Best val accuracy: 0.076923079788684

2600/2600 [==============================] - 10s 4ms/step - loss: 2.6189 - accuracy: 0.0769 - val_loss: 2.6187 - val_accuracy: 0.0769
Best val accuracy: 0.07692307978868484
Training model 8 with 1993 params - (4, 4, 4, 4); 4; 0.005; 0.5; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 11s 4ms/step - loss: 2.6188 - accuracy: 0.0762 - val_loss: 2.6187 - val_accuracy: 0.0769
Epoch 2/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6185 - accuracy: 0.0763 - val_loss: 2.6189 - val_accuracy: 0.0769
Epoch 3/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6187 - accuracy: 0.0760 - val_loss: 2.6183 - val_accuracy: 0.0769
Epoch 4/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6186 - accuracy: 0.0752 - val_loss: 2.6183 - val_accuracy: 0.0769
Epoch 5/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.6186 - accuracy: 0.0789 - val_loss: 2.6183 - val_accuracy: 0.0769
Epoch 6/

2600/2600 [==============================] - 947s 364ms/step - loss: 77.7972 - accuracy: 0.1651 - val_loss: 14.8052 - val_accuracy: 0.1457
Epoch 4/50
2600/2600 [==============================] - 2072s 797ms/step - loss: 9.7299 - accuracy: 0.1650 - val_loss: 5.8947 - val_accuracy: 0.1698
Epoch 5/50
2600/2600 [==============================] - 2004s 771ms/step - loss: 555.7085 - accuracy: 0.1421 - val_loss: 4698.2744 - val_accuracy: 0.1084
Epoch 6/50
2600/2600 [==============================] - 160s 61ms/step - loss: 7419.9256 - accuracy: 0.1335 - val_loss: 8942.5088 - val_accuracy: 0.1446
Epoch 7/50
2600/2600 [==============================] - 10s 4ms/step - loss: 9080.9362 - accuracy: 0.1363 - val_loss: 6569.7637 - val_accuracy: 0.1535
Best val accuracy: 0.21067307889461517
Training model 15 with 1993 params - (4, 4, 4, 4); 4; 0.0005; 0.2; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 11s 4ms/step - loss: 6.5575 - accuracy: 0.1439 - val_loss: 467.

Epoch 7/50
2600/2600 [==============================] - 10s 4ms/step - loss: 3.3033 - accuracy: 0.1594 - val_loss: 3.1368 - val_accuracy: 0.1775
Best val accuracy: 0.20538461208343506
Training model 20 with 1993 params - (4, 4, 4, 4); 4; 0.0001; 0.5; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 11s 4ms/step - loss: 2.6124 - accuracy: 0.0911 - val_loss: 2.5282 - val_accuracy: 0.1706
Epoch 2/50
2600/2600 [==============================] - 10s 4ms/step - loss: 8.1556 - accuracy: 0.1162 - val_loss: 376.2892 - val_accuracy: 0.1699
Epoch 3/50
2600/2600 [==============================] - 10s 4ms/step - loss: 1101.3648 - accuracy: 0.0800 - val_loss: 3649.4941 - val_accuracy: 0.1280
Epoch 4/50
2600/2600 [==============================] - 10s 4ms/step - loss: 3147.5372 - accuracy: 0.0799 - val_loss: 2551.4065 - val_accuracy: 0.1282
Epoch 5/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2647.2666 - accuracy: 0.0780 - val_loss: 4011.811

2600/2600 [==============================] - 16s 6ms/step - loss: 257538209.2681 - accuracy: 0.1578 - val_loss: 349946048.0000 - val_accuracy: 0.1631
Epoch 12/50
2600/2600 [==============================] - 16s 6ms/step - loss: 370317400.4015 - accuracy: 0.1608 - val_loss: 643706368.0000 - val_accuracy: 0.1644
Epoch 13/50
2600/2600 [==============================] - 16s 6ms/step - loss: 648456519.9982 - accuracy: 0.1619 - val_loss: 688295552.0000 - val_accuracy: 0.1661
Epoch 14/50
2600/2600 [==============================] - 16s 6ms/step - loss: 694211434.6482 - accuracy: 0.1616 - val_loss: 593350400.0000 - val_accuracy: 0.1667
Best val accuracy: 0.16865384578704834
Training model 26 with 7717 params - (8, 8, 8, 8); 8; 0.005; 0.1; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 17s 6ms/step - loss: 3502.2012 - accuracy: 0.1927 - val_loss: 395521.3125 - val_accuracy: 0.1587
Epoch 2/50
2600/2600 [==============================] - 16s 6ms/step - loss: 

...

Epoch 13/50
2600/2600 [==============================] - 46s 18ms/step - loss: 6142139732445.1582 - accuracy: 0.1190 - val_loss: 5275438284800.0000 - val_accuracy: 0.1303
Best val accuracy: 0.1526923030614853
Training model 69 with 120685 params - (32, 32, 32, 32); 32; 0.001; 0.05; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 47s 18ms/step - loss: 1.6657 - accuracy: 0.4883 - val_loss: 0.8848 - val_accuracy: 0.7518
Epoch 2/50
2600/2600 [==============================] - 46s 18ms/step - loss: 0.8955 - accuracy: 0.7410 - val_loss: 0.7459 - val_accuracy: 0.8040
Epoch 3/50
2600/2600 [==============================] - 47s 18ms/step - loss: 0.7611 - accuracy: 0.7821 - val_loss: 0.6720 - val_accuracy: 0.8163
Epoch 4/50
2600/2600 [==============================] - 47s 18ms/step - loss: 0.7073 - accuracy: 0.8013 - val_loss: 0.6406 - val_accuracy: 0.8214
Epoch 5/50
2600/2600 [==============================] - 47s 18ms/step - loss: 0.6433 - accuracy: 0.8201 

2600/2600 [==============================] - 46s 18ms/step - loss: 1919737895.6155 - accuracy: 0.1546 - val_loss: 5588957184.0000 - val_accuracy: 0.2002
Best val accuracy: 0.2566346228122711
Training model 73 with 120685 params - (32, 32, 32, 32); 32; 0.0005; 0.05; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 50s 19ms/step - loss: 1.8024 - accuracy: 0.4516 - val_loss: 1.0509 - val_accuracy: 0.7008
Epoch 2/50
2600/2600 [==============================] - 48s 18ms/step - loss: 1.0764 - accuracy: 0.6892 - val_loss: 0.9707 - val_accuracy: 0.7296
Epoch 3/50
2600/2600 [==============================] - 47s 18ms/step - loss: 0.9083 - accuracy: 0.7331 - val_loss: 0.8032 - val_accuracy: 0.7746
Epoch 4/50
2600/2600 [==============================] - 49s 19ms/step - loss: 0.8160 - accuracy: 0.7605 - val_loss: 0.7879 - val_accuracy: 0.7810
Epoch 5/50
2600/2600 [==============================] - 48s 19ms/step - loss: 0.7614 - accuracy: 0.7775 - val_loss: 0.696

2600/2600 [==============================] - 45s 17ms/step - loss: 965368.6934 - accuracy: 0.2372 - val_loss: 1124159.8750 - val_accuracy: 0.2882
Epoch 9/50
2600/2600 [==============================] - 45s 17ms/step - loss: 1783875.9981 - accuracy: 0.2387 - val_loss: 2727086.7500 - val_accuracy: 0.3294
Epoch 10/50
2600/2600 [==============================] - 45s 17ms/step - loss: 3446265.1012 - accuracy: 0.2353 - val_loss: 4259280.5000 - val_accuracy: 0.2521
Epoch 11/50
2600/2600 [==============================] - 45s 17ms/step - loss: 6299584.8601 - accuracy: 0.2299 - val_loss: 6359548.5000 - val_accuracy: 0.2906
Epoch 12/50
2600/2600 [==============================] - 45s 17ms/step - loss: 11711559.1077 - accuracy: 0.2222 - val_loss: 13386259.0000 - val_accuracy: 0.3025
Best val accuracy: 0.330240398645401
Training model 77 with 120685 params - (32, 32, 32, 32); 32; 0.0001; 0.05; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 47s 18ms/step - loss

2600/2600 [==============================] - 108s 42ms/step - loss: 2.7856 - accuracy: 0.0761 - val_loss: 2.7850 - val_accuracy: 0.0769
Epoch 5/50
2600/2600 [==============================] - 108s 42ms/step - loss: 2.7855 - accuracy: 0.0766 - val_loss: 2.7855 - val_accuracy: 0.0769
Epoch 6/50
2600/2600 [==============================] - 108s 42ms/step - loss: 2.7856 - accuracy: 0.0769 - val_loss: 2.7848 - val_accuracy: 0.0769
Best val accuracy: 0.07692307978868484
Training model 82 with 480973 params - (64, 64, 64, 64); 64; 0.01; 0.1; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 111s 42ms/step - loss: 2.3007 - accuracy: 0.3382 - val_loss: 1.7609 - val_accuracy: 0.5656
Epoch 2/50
2600/2600 [==============================] - 109s 42ms/step - loss: 2.7724 - accuracy: 0.4530 - val_loss: 95.7121 - val_accuracy: 0.2089
Epoch 3/50
2600/2600 [==============================] - 109s 42ms/step - loss: 162228.8020 - accuracy: 0.1595 - val_loss: 4462691.0000 

KeyboardInterrupt: 

# Evaluation of the best model

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

Training and validation performances over epochs

In [ ]:
plt.plot(best_history.history['loss'])
plt.plot(best_history.history['val_loss'])
plt.title('Convergence of the loss function')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train','val'])

In [ ]:
plt.plot(best_history.history['accuracy'])
plt.plot(best_history.history['val_accuracy'])
plt.title('Convergence of the accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train','val'])

Confusion matrix

In [ ]:
plot_conf_mtx(best_model, X_val, y_val, df_val)